In [ ]:
import turicreate as tc
import pandas as pd
import json
import numpy as np
import urllib.request
from PIL import Image
import numpy as np
import urllib
import cv2

In [ ]:
cd "Taco"

In [ ]:
path = "/Users/mithurangajendran/Documents/X/Cours/P3/SURFRIDER/Taco"
dataset_path = path
anns_file_path = dataset_path + '/' + 'annotations.json'

# Read annotations
with open(anns_file_path, 'r') as f:
    dataset = json.loads(f.read())
    
categories = dataset['categories']
anns = dataset['annotations']
imgs = dataset['images']
nr_cats = len(categories)
nr_annotations = len(anns)
nr_images = len(imgs)

# Load categories and super categories
cat_names = []
super_cat_names = []
super_cat_ids = {}
super_cat_last_name = ''
nr_super_cats = 0
for cat_it in categories:
    cat_names.append(cat_it['name'])
    super_cat_name = cat_it['supercategory']
    # Adding new supercat
    if super_cat_name != super_cat_last_name:
        super_cat_names.append(super_cat_name)
        super_cat_ids[super_cat_name] = nr_super_cats
        super_cat_last_name = super_cat_name
        nr_super_cats += 1

print('Number of super categories:', nr_super_cats)
print('Number of categories:', nr_cats)
print('Number of annotations:', nr_annotations)
print('Number of images:', nr_images)

In [ ]:
dict_categorizer = \
{'Aluminium foil':"fragments",
 'Battery':"others",
 'Blister pack':"fragments",
 'Bottle':"bottles",
 'Bottle cap':"fragments",
 'Broken glass':"fragments",
 'Can':"others",
 'Carton':"others",
 'Cigarette':"others",
 'Cup':"fragments",
 'Food waste':"others",
 'Glass jar':"others",
 'Lid':"fragments",
 'Other plastic':"fragments",
 'Paper':"others",
 'Paper bag':"fragments",
 'Plastic bag & wrapper':"fragments",
 'Plastic container':"fragments",
 'Plastic glooves':"fragments",
 'Plastic utensils':"fragments",
 'Pop tab':"others",
 'Rope & strings':"fragments",
 'Scrap metal':"others",
 'Shoe':"others",
 'Squeezable tube':"fragments",
 'Straw':"fragments",
 'Styrofoam piece':"fragments",
 'Unlabeled litter':"fragments"}

def bucketer(category):
    try:
        return dict_categorizer[category]
    except:
        return "others"
    #return 

In [ ]:
cat_1 = pd.DataFrame.from_dict(categories, orient='columns').drop(columns="name")
anns_1 = pd.DataFrame.from_dict(anns, orient='columns').drop(columns=["segmentation","area","iscrowd"])
imgs_1 = pd.DataFrame.from_dict(imgs, orient='columns').drop(columns=["file_name","license",
                                       "coco_url","date_captured"])

merged = anns_1.merge(imgs_1, left_on="image_id", right_on="id")
merged = merged.merge(cat_1, left_on="category_id",right_on="id")
merged = merged.drop(columns=["category_id","id_y","id"])

merged.supercategory = merged.supercategory.apply(lambda category: bucketer(category))

In [ ]:
#merged = merged[:100]

In [ ]:
def url_to_image(url):
    resp = urllib.request.urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    image = Image.fromarray(image)
    return image

def clean_annotations(line):

    label = line['supercategory']
    coordinates = line['bbox']
    coordinates = {"x": int(coordinates[0]),
                   "y": int(coordinates[1]),
                   "width": int(coordinates[2]),
                   "height": int(coordinates[3])}
    return {"label":label, "coordinates":coordinates}

def dl_img(url):
    try:
        return tc.Image(url)
        #return url_to_image(url)
    except:
        return np.nan
    
def label_sizer(line):
    L=[]
    for i in range(len(line["annotations"])):
        label = line["annotations"][i]["label"]
        coordinates = line["annotations"][i]["coordinates"]
        
        r_height = line.flickr_640_url.height / line["height"]
        r_width = line.flickr_640_url.width / line["width"]
        
        coordinate = {"x": r_width*(int(coordinates["x"])+float(coordinates["width"])/2),
                   "y": r_height*(int(coordinates["y"])+float(coordinates["height"])/2),
                   "height": r_height*int(coordinates["height"]),
                  "width": r_width*int(coordinates["width"])}
        L.append({"label":label, "coordinates":coordinate})
    return L

In [ ]:
merged["annotations"] = merged.apply(lambda line: clean_annotations(line), 
                                     axis=1)

merged = merged.drop(columns=["bbox","supercategory","id_x"])
details_to_merge = merged[["image_id","width","height"]].drop_duplicates().copy()

merged = pd.DataFrame(merged.groupby(["image_id",
        "flickr_640_url"]).apply(lambda x: list(x.annotations))).reset_index("flickr_640_url")\
    .rename(columns = {0:"annotations"})

merged['flickr_640_url'] = merged['flickr_640_url'].apply(lambda x: dl_img(str(x)))

merged = merged.reset_index().merge(details_to_merge, on="image_id", how="inner")

merged["annotations"] = merged.apply(lambda x: label_sizer(x),axis=1)

In [ ]:
SF = tc.SFrame(data=merged,format="dataframe").rename({'flickr_640_url': 'image'})

In [ ]:
SF.remove_columns(column_names=["image_id","width","height"], inplace=True)

In [ ]:
SF['image_with_ground_truth'] = tc.object_detector.util.draw_bounding_boxes(SF['image'], SF['annotations'])
#SF.explore("epic")

In [ ]:
SF.save("to_load_Taco")
#Taco = tc.load_sframe("to_load_Taco")
#SF.append(Taco).tail()

In [ ]:
SF.explore()